In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import nltk

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
train=pd.read_csv("bases/working_train.csv",encoding="utf8")
test=pd.read_csv("bases/working_test.csv",encoding="utf8")

In [3]:
#LEAK PART FOR EVALUATING FEATURES :
###### OR USE THIS IF U WANT TO USE THE LEAKS!!!!! (not necessary!) ##############################s
pickle_in = open("Leaks/Leaks_all", "rb") #### OR use Leaks for the full leaks (no sub)
Leaks = pickle.load(pickle_in)

test_leak = Leaks
test_leak.index = test_leak.ID
Class_leaks = test_leak.Class
test =test[test.ID.isin(test_leak.index)] # drop Class, to be used for validation later on 

In [4]:
def evaluate_features(X_train,y_train,X_test,clf=None):
    if clf is None:
        clf = LogisticRegression()
    clf.fit(X_train,y_train)
    probas=clf.predict_proba(X_test)
    y = Class_leaks - 1
    print('Log loss: {}'.format(log_loss(y, probas)))

In [5]:
y_train=np.array(train["Class"])-1

In [8]:
#this function can be used to check if one variable could be useful. 
evaluate_features(X_train,y_train,X_test)

Log loss: 1.6028411177680808


# Here we will add features from word2vec retrained then get the mean for the sentences

In [9]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [11]:
#It's important to remove duplicated spaces for word2vec learning !
train["Text"]=[" ".join(doc.split()) for doc in train["Text"].values]
test["Text"]=[" ".join(doc.split()) for doc in test["Text"].values]

In [13]:
w2vec = get_word2vec(
    MySentences(
        train["Text"].values,test["Text"].values),"localisation"
)

localisation not found. training model
Model done training. Saving to disk


In [16]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [44]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded_train = mean_embedding_vectorizer.fit_transform(train['Text'])
mean_embedded_test = mean_embedding_vectorizer.fit_transform(test['Text'])

In [45]:
df_embed_tr=pd.DataFrame(mean_embedded_train)
df_embed_te=pd.DataFrame(mean_embedded_test)

In [46]:
df_embedding_tr=df_embed_tr.reset_index()
df_embedding_te=df_embed_te.reset_index()
df_embedding_tr=df_embedding_tr.rename(columns={"index":"ID"})
df_embedding_te=df_embedding_te.rename(columns={"index":"ID"})

In [49]:
train_w2v=pd.merge(train,df_embedding_tr,on="ID")
test_w2v=pd.merge(test,df_embedding_te,on="ID")

In [54]:
test_w2v=test_w2v.drop("Class",axis=1)

In [56]:
train_w2v.to_csv("bases/working_train_w2v.csv",index=False,encoding="utf8")
test_w2v.to_csv("bases/working_test_w2v.csv",index=False,encoding="utf8")

# Now TFIDF

# THEN FASTTEXT